### Overwiew
The primary objective of this project is to analyze the factors contributing to the decline in sales for SalesEase, an online retailer, and provide insights to reverse this trend. 


**The isuue: Declining Sales**
- Six months of continuous decline in sales.
- Concerns about the sustainability of the business.
- Need to understand underlying factors affecting sales performance.


**Solution: Conduct Exploratory Data Analysis (EDA) to uncover insights**

The key areas of focus include identifying product categories' performance, understanding customer behavior, evaluating sales channels' effectiveness, and optimizing various aspects of the e-commerce operation.

### Project Steps
1. Data Exploration
2. Time Series Analysis
3. Customer Segmentation Analysis
4. Product Analysis
5. Payment and Returns Analysis
6. Insights and Recommendations